이렇게 하려면 케라스 모델을 사이킷런 추정기처럼 보이게 바꿔야합니다.<br>
먼저 일련의 하이퍼파라미터로 케라스 모델을 만들고 컴파일하는 함수를 만듭니다.

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [2]:
np.random.seed(42)
tf.random.set_seed(42)

In [3]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [4]:
housing = fetch_california_housing()

In [5]:
X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

X_new = X_test[:3]

In [6]:
def build_model(n_hidden=1, n_neurons=30, learning_rate=3e-3, input_shape=[8]):
    model = keras.models.Sequential()
    model.add(keras.layers.InputLayer(input_shape=input_shape))
    for layer in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons, activation='relu'))
    model.add(keras.layers.Dense(1))
    optimizer = keras.optimizers.SGD(learning_rate=learning_rate)
    model.compile(loss='mse', optimizer=optimizer)
    return model

build_model() 함수를 사용해 KerasRegressor 클래스 객체를 만듭니다.

In [7]:
keras_reg = keras.wrappers.scikit_learn.KerasRegressor(build_model)

C:\Users\jiweo\AppData\Local\Temp/ipykernel_30960/1709004121.py:1: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  keras_reg = keras.wrappers.scikit_learn.KerasRegressor(build_model)


이 객체는 build_model() 함수로 만들어진 케라스 모델을 감싸는 간단한 래퍼입니다.<br>
이 객체를 만들 때 어떤 하이퍼파라미터도 정의하지 않았으므로 build_model()에 정의된 기본 파라미터를 사용할 것입니다.

이제 일반적 사이킷런 회귀 추정기처럼 이 객체를 사용할 수 있습니다.

In [8]:
keras_reg.fit(X_train, y_train, epochs=100,
             validation_data=(X_valid, y_valid),
             callbacks=[keras.callbacks.EarlyStopping(patience=10)])
mse_test = keras_reg.score(X_test, y_test)
y_pred = keras_reg.predict(X_new)

Epoch 1/100
363/363 [==============================] - 1s 2ms/step - loss: 1.0896 - val_loss: 20.7721
Epoch 2/100
363/363 [==============================] - 1s 2ms/step - loss: 0.7606 - val_loss: 5.0266
Epoch 3/100
363/363 [==============================] - 1s 1ms/step - loss: 0.5456 - val_loss: 0.5490
Epoch 4/100
363/363 [==============================] - 1s 2ms/step - loss: 0.4732 - val_loss: 0.4529
Epoch 5/100
363/363 [==============================] - 1s 2ms/step - loss: 0.4503 - val_loss: 0.4188
Epoch 6/100
363/363 [==============================] - 1s 2ms/step - loss: 0.4338 - val_loss: 0.4129
Epoch 7/100
363/363 [==============================] - 1s 2ms/step - loss: 0.4241 - val_loss: 0.4004
Epoch 8/100
363/363 [==============================] - 1s 2ms/step - loss: 0.4168 - val_loss: 0.3944
Epoch 9/100
363/363 [==============================] - 1s 2ms/step - loss: 0.4108 - val_loss: 0.3961
Epoch 10/100
363/363 [==============================] - 1s 2ms/step - loss: 0.4060 - val_l

fit() 메소드에 지정한 모든 매개변수는 케라스 모델로 전달됩니다.<br>
사이킷런은 손실이 아니라 점수를 계산하기 때문에 출력 점수는 음수의 MSE입니다.

모델 하나를 훈련하고 평가하려는 게 아니라 수백 개의 모델을 훈련하고<br>
검증 세트에서 최상의 성능을 보이는 모델을 선택해야 합니다.

하이퍼파라미터가 많으므로 그리드탐색보다 랜덤 탐색을 사용하는 게 좋습니다.<br>
은닉층 개수, 뉴런 개수, 학습률을 사용해 하이퍼파라미터 탐색을 해보겠습니다.

In [9]:
from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV

In [10]:
param_distribs = {
    'n_hidden': [0, 1, 2, 3],
    'n_neurons': np.arange(1, 100),
    'learning_rate': reciprocal(3e-4, 3e-2)
}

rnd_search_cv = RandomizedSearchCV(keras_reg, param_distribs, n_iter=10, cv=3, verbose=2)
rnd_search_cv.fit(X_train, y_train, epochs=100,
                 validation_data=(X_valid, y_valid),
                 callbacks=[keras.callbacks.EarlyStopping(patience=10)])

Fitting 3 folds for each of 10 candidates, totalling 30 fits
Epoch 1/100
242/242 [==============================] - 1s 3ms/step - loss: 3.3071 - val_loss: 1.3735
Epoch 2/100
242/242 [==============================] - ETA: 0s - loss: 1.005 - 0s 2ms/step - loss: 0.9782 - val_loss: 0.6881
Epoch 3/100
242/242 [==============================] - 0s 2ms/step - loss: 0.6186 - val_loss: 0.5887
Epoch 4/100
242/242 [==============================] - 0s 2ms/step - loss: 0.5504 - val_loss: 0.5466
Epoch 5/100
242/242 [==============================] - 0s 2ms/step - loss: 0.5363 - val_loss: 0.5360
Epoch 6/100
242/242 [==============================] - 0s 2ms/step - loss: 0.5307 - val_loss: 0.7261
Epoch 7/100
242/242 [==============================] - 0s 2ms/step - loss: 0.5322 - val_loss: 0.6728
Epoch 8/100
242/242 [==============================] - 0s 2ms/step - loss: 0.5311 - val_loss: 0.6939
Epoch 9/100
242/242 [==============================] - 0s 2ms/step - loss: 0.5326 - val_loss: 0.5351
Epoch 

242/242 [==============================] - 0s 2ms/step - loss: 127.6586 - val_loss: 23472.3613
Epoch 7/100
242/242 [==============================] - 0s 2ms/step - loss: 242.6267 - val_loss: 91941.1250
Epoch 8/100
242/242 [==============================] - 0s 2ms/step - loss: 1803.9309 - val_loss: 363473.0312
Epoch 9/100
242/242 [==============================] - 0s 2ms/step - loss: 4479.3096 - val_loss: 1426668.5000
Epoch 10/100
242/242 [==============================] - 0s 2ms/step - loss: 31568.9824 - val_loss: 5931677.5000
Epoch 11/100
121/121 [==============================] - 0s 1ms/step - loss: 61659.1211
[CV] END learning_rate=0.008731907739399206, n_hidden=0, n_neurons=21; total time=   5.1s
Epoch 1/100
242/242 [==============================] - 1s 2ms/step - loss: 1.4335 - val_loss: 4.8167
Epoch 2/100
242/242 [==============================] - 0s 2ms/step - loss: 0.5960 - val_loss: 14.1333
Epoch 3/100
242/242 [==============================] - 0s 2ms/step - loss: 0.5457 - val

Epoch 48/100
242/242 [==============================] - 0s 2ms/step - loss: 0.3825 - val_loss: 0.3844
Epoch 49/100
242/242 [==============================] - 0s 2ms/step - loss: 0.3812 - val_loss: 0.3851
Epoch 50/100
242/242 [==============================] - 0s 2ms/step - loss: 0.3799 - val_loss: 0.3993
Epoch 51/100
242/242 [==============================] - 0s 2ms/step - loss: 0.3787 - val_loss: 0.3877
Epoch 52/100
242/242 [==============================] - 1s 2ms/step - loss: 0.3779 - val_loss: 0.3881
Epoch 53/100
121/121 [==============================] - 0s 1ms/step - loss: 0.3925
[CV] END learning_rate=0.0006154014789262348, n_hidden=2, n_neurons=87; total time=  27.7s
Epoch 1/100
242/242 [==============================] - 1s 2ms/step - loss: 2.5809 - val_loss: 4.4401
Epoch 2/100
242/242 [==============================] - 1s 2ms/step - loss: 0.9454 - val_loss: 5.4218
Epoch 3/100
242/242 [==============================] - 1s 2ms/step - loss: 0.7935 - val_loss: 4.0185
Epoch 4/100
2

242/242 [==============================] - 0s 2ms/step - loss: 0.3693 - val_loss: 0.3644
Epoch 52/100
242/242 [==============================] - 0s 2ms/step - loss: 0.3683 - val_loss: 0.3684
Epoch 53/100
242/242 [==============================] - 0s 2ms/step - loss: 0.3673 - val_loss: 0.3565
Epoch 54/100
242/242 [==============================] - 0s 2ms/step - loss: 0.3663 - val_loss: 0.3509
Epoch 55/100
242/242 [==============================] - 0s 2ms/step - loss: 0.3652 - val_loss: 0.3532
Epoch 56/100
242/242 [==============================] - 1s 2ms/step - loss: 0.3641 - val_loss: 0.3614
Epoch 57/100
242/242 [==============================] - 1s 2ms/step - loss: 0.3631 - val_loss: 0.3659
Epoch 58/100
242/242 [==============================] - 0s 2ms/step - loss: 0.3625 - val_loss: 0.3618
Epoch 59/100
242/242 [==============================] - 0s 2ms/step - loss: 0.3615 - val_loss: 0.3671
Epoch 60/100
242/242 [==============================] - 0s 2ms/step - loss: 0.3605 - val_loss: 

Epoch 47/100
242/242 [==============================] - 0s 2ms/step - loss: 0.4444 - val_loss: 0.4148
Epoch 48/100
242/242 [==============================] - 0s 2ms/step - loss: 0.4420 - val_loss: 0.4132
Epoch 49/100
242/242 [==============================] - 0s 2ms/step - loss: 0.4400 - val_loss: 0.4121
Epoch 50/100
242/242 [==============================] - 0s 2ms/step - loss: 0.4379 - val_loss: 0.4107
Epoch 51/100
242/242 [==============================] - 0s 2ms/step - loss: 0.4358 - val_loss: 0.4084
Epoch 52/100
242/242 [==============================] - 1s 2ms/step - loss: 0.4342 - val_loss: 0.4088
Epoch 53/100
242/242 [==============================] - 0s 2ms/step - loss: 0.4323 - val_loss: 0.4082
Epoch 54/100
242/242 [==============================] - 0s 2ms/step - loss: 0.4304 - val_loss: 0.4057
Epoch 55/100
242/242 [==============================] - 0s 2ms/step - loss: 0.4285 - val_loss: 0.4059
Epoch 56/100
242/242 [==============================] - 0s 2ms/step - loss: 0.4270

...

242/242 [==============================] - 0s 2ms/step - loss: 0.5458 - val_loss: 21.4883
Epoch 76/100
242/242 [==============================] - 0s 2ms/step - loss: 0.5450 - val_loss: 21.4613
Epoch 77/100
242/242 [==============================] - 0s 2ms/step - loss: 0.5442 - val_loss: 21.4306
Epoch 78/100
242/242 [==============================] - 0s 2ms/step - loss: 0.5435 - val_loss: 21.4049
Epoch 79/100
242/242 [==============================] - 0s 1ms/step - loss: 0.5427 - val_loss: 21.3768
Epoch 80/100
242/242 [==============================] - 0s 2ms/step - loss: 0.5420 - val_loss: 21.3550
Epoch 81/100
242/242 [==============================] - 0s 2ms/step - loss: 0.5412 - val_loss: 21.3345
Epoch 82/100
242/242 [==============================] - 0s 2ms/step - loss: 0.5405 - val_loss: 21.3160
Epoch 83/100
242/242 [==============================] - 0s 2ms/step - loss: 0.5398 - val_loss: 21.2926
Epoch 84/100
242/242 [==============================] - 0s 2ms/step - loss: 0.5391 - v

Epoch 6/100
242/242 [==============================] - 1s 2ms/step - loss: 0.4097 - val_loss: 0.3831
Epoch 7/100
242/242 [==============================] - 1s 2ms/step - loss: 0.3999 - val_loss: 0.4131
Epoch 8/100
242/242 [==============================] - 0s 2ms/step - loss: 0.3921 - val_loss: 0.4451
Epoch 9/100
242/242 [==============================] - 0s 2ms/step - loss: 0.3853 - val_loss: 0.5546
Epoch 10/100
242/242 [==============================] - 0s 2ms/step - loss: 0.3808 - val_loss: 0.5930
Epoch 11/100
242/242 [==============================] - 1s 2ms/step - loss: 0.3757 - val_loss: 0.6783
Epoch 12/100
242/242 [==============================] - 0s 2ms/step - loss: 0.3713 - val_loss: 0.7282
Epoch 13/100
242/242 [==============================] - 0s 2ms/step - loss: 0.3690 - val_loss: 0.8016
Epoch 14/100
242/242 [==============================] - ETA: 0s - loss: 0.363 - 0s 2ms/step - loss: 0.3663 - val_loss: 0.9122
Epoch 15/100
242/242 [==============================] - 0s 2ms

242/242 [==============================] - 1s 2ms/step - loss: 0.3192 - val_loss: 0.3305
Epoch 69/100
242/242 [==============================] - 0s 2ms/step - loss: 0.3189 - val_loss: 0.3093
Epoch 70/100
242/242 [==============================] - 0s 2ms/step - loss: 0.3180 - val_loss: 0.3264
Epoch 71/100
242/242 [==============================] - 0s 2ms/step - loss: 0.3186 - val_loss: 0.3097
Epoch 72/100
242/242 [==============================] - 1s 2ms/step - loss: 0.3183 - val_loss: 0.3093
Epoch 73/100
242/242 [==============================] - 0s 2ms/step - loss: 0.3172 - val_loss: 0.3105
Epoch 74/100
242/242 [==============================] - 0s 2ms/step - loss: 0.3180 - val_loss: 0.3071
Epoch 75/100
242/242 [==============================] - 0s 2ms/step - loss: 0.3157 - val_loss: 0.3233
Epoch 76/100
242/242 [==============================] - 0s 2ms/step - loss: 0.3141 - val_loss: 0.3508
Epoch 77/100
242/242 [==============================] - 0s 2ms/step - loss: 0.3153 - val_loss: 

363/363 [==============================] - 1s 2ms/step - loss: 0.3255 - val_loss: 0.3235
Epoch 47/100
363/363 [==============================] - 1s 2ms/step - loss: 0.3257 - val_loss: 0.3443
Epoch 48/100
363/363 [==============================] - 1s 2ms/step - loss: 0.3244 - val_loss: 0.3226
Epoch 49/100
363/363 [==============================] - 1s 2ms/step - loss: 0.3253 - val_loss: 0.3215
Epoch 50/100
363/363 [==============================] - 1s 2ms/step - loss: 0.3228 - val_loss: 0.3138
Epoch 51/100
363/363 [==============================] - 1s 2ms/step - loss: 0.3226 - val_loss: 0.3494
Epoch 52/100
363/363 [==============================] - 1s 2ms/step - loss: 0.3220 - val_loss: 0.3629
Epoch 53/100
363/363 [==============================] - 1s 2ms/step - loss: 0.3213 - val_loss: 0.4480
Epoch 54/100
363/363 [==============================] - 1s 2ms/step - loss: 0.3225 - val_loss: 0.4090
Epoch 55/100
363/363 [==============================] - 1s 2ms/step - loss: 0.3206 - val_loss: 

RandomizedSearchCV(cv=3,
                   estimator=<keras.wrappers.scikit_learn.KerasRegressor object at 0x000001EBBF39C0D0>,
                   param_distributions={'learning_rate': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001EBC0B94610>,
                                        'n_hidden': [0, 1, 2, 3],
                                        'n_neurons': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
       69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85,
       86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99])},
                   verbose=2)

RandomizedSearchCV는 k-겹 교차 검증을 사용하기 때문에 X_valid와 y_valid를 사용하지 않습니다.<br>
이 데이터는 조기 종료 시에만 사용됩니다.

다음은 하이퍼파라미터 최적화에 사용할 수 있는 다른 파이썬 라이브러리들입니다.

- Hyperopt
- Hyperas, kopt, Talos
- Keras Tuner
- Scikit-Optimize(skopt)
- Spearmint
- Hyperband
- Sklearn-Deap